Import required modules:

In [1]:
"""
Interactive system simulation using ipywidgets package.
Visualization functions used in this program are in the TRPL.sim.vizfuncs module.
@author: Natalia Spitha
"""
import os
import numpy as np

from KinetiKit import sim, data
from KinetiKit import artists as art
from KinetiKit import kit as kin_kit
from KinetiKit.units import units, ps, ns

from ipywidgets import interact, fixed

## Import Data
Recommended instructions: 
<br>
- Define time parameters in the time object to (see KinetiKit.sim.time for more information)
- Set the path directory _dir_path_ to a general folder where your files are located.
- Define the file names (including extension) and, if necessary, other subfolders that determine _file_path_.
- Define power unit and a list of powers of equal length to the list of file names. See TRPL.units for list of units.
- Ensure your files are imported correctly (check _data_from_SPCM_ in TRPL.data.lib)
<br> <br>
Optional: 
- Define directory for file containing dark counts (_dark_path_)
- Un-comment _do.dark_subtract, do.smooth,_ and _do.remove_zeros()_ if you want to apply those functions.

In [2]:
N_timepoints = 1000
to = sim.time.linear(N=N_timepoints, period = 12.5*ns, subsample=1)
dtime = to['array'][::to['subsample']]

#--- Creating Data Object(s)
all_data = []; keys = []
dir_path = os.getcwd()
#dir_path = r'C:\...'
dark_path = os.path.join(dir_path, 'ex_data', 'dark.asc')

filenames = [r'low_power.asc', r'mid_power.asc', r'high_power.asc',]

all_y = np.zeros((len(filenames), len(dtime))) # y data will eventually be appended to this list
power_unit = 'μW'
powers = [0.045, 0.420, 1.015] # excitation powers in defined unit

for i, filename in enumerate(filenames):
    subfolder = filename.split('_')[0] # using information from the file name to determine which folder it's in
    file_path = os.path.join(dir_path, 'ex_data', subfolder, filename)
    do = data.lib.data_from_SPCM(file_path, skip_h=354, skip_f=884, metadata=True)
    do.interp(dtime)
    all_y[i] = do.y

aligned_data = kin_kit.align_by_steep(all_y, dtime, value = 0.5*ns, avgnum=5)

## Define system and parameters
Instructions: 
<br>
- Select your system: _system = sim.systems.YourSystem()_
- Select whether simulations will use the _powers_ defined above (slidepower=False), or a power slider (slidepower=True)
- Define parameters as a dictionary, where the "values" are actually widget objects. You can customize the values, limits, step sizes, etc. for each slider object. <font color=red> Note: the interactive visualization function currently accepts up to 20 model parameters. </font>
<br> <br>
Optional: 
- If _slidepower_ is True, customize the power widget; see TRPL.sim.vizfuncs.powerwidget for details.

In [3]:
system = sim.systems.Biexp()

slidepower = False # select whether to have a slider controlling power

# Define all the parameters of the system chosen. 
# You can call the function system.params() on a cell below to quickly check which parameters are needed.

parameters = {
    
    'A1' : sim.vizfuncs.makeWidget('A1', val=0.5, slider='linear', min_val=0, max_val=1),
                  
    'tau1' : sim.vizfuncs.makeWidget('tau1', val=1.4e-9, slider='log', min_val=1e-10, max_val=1e-6),
    
    'tau2' : sim.vizfuncs.makeWidget('tau2', val=1e-7, slider='log', min_val=1e-10, max_val=1e-6),
    
    'offset' : sim.vizfuncs.makeWidget('offset', val=0.5, slider='linear', min_val=0, max_val=1),
    
        }

active_widgets = sim.vizfuncs.active_widgets(system, parameters)

w1, w2, w3, w4, w5, w6, w7, w8, w9, w10,\
w11, w12, w13, w14, w15, w16, w17, w18, w19, w20 = active_widgets # 20 widgets are needed as input to the Viz function, 
                                                                  # but only the active widgets will be used

powerwidget = sim.vizfuncs.powerwidget(slidepower=slidepower, unit=power_unit)

## Interactive Plot
<br>
Your interactive plot is ready (hopefully!)
<br>
Optional instructions:
- Adjust other arguments accepted by the MonoViz function, such as N_coarse. 
<br>
If the message "Array does not contain positive values!" appears, it is likely that your parameter is too fast for the chosen step size. You can increase N_coarse, as well as the N argument of the time object in the data import cell, to account for faster processes.

In [4]:
print('%s simulation:'%system.name)
GenWidget = interact(sim.vizfuncs.FuncViz,
                     system = fixed(system),
                     p1=w1, p2=w2, p3=w3, p4=w4, p5=w5, p6=w6, p7=w7, p8=w8, p9=w9, p10=w10,
                     p11=w11, p12=w12, p13=w13, p14=w14, p15=w15, p16=w16, p17=w17, p18=w18,
                     p19=w19, p20=w20,
                     to=fixed(to),
                     N_coarse = fixed(N_timepoints),
                     power = powerwidget,
                     data=fixed(aligned_data),
                     power_list = fixed(powers),
                     power_unit = fixed(power_unit),
                     align_by = fixed('steep'),
                     avgnum = fixed(5),
                     slidepower=fixed(slidepower)
                     )

Biexp simulation:


interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='A1', max=1.0, step=0.01), F…